This function computes the interpolant for the instantaneous momentum barrier equation:

 \begin{equation}
 \mathbf{x}'=\nu \rho \mathbf{J} \mathbf{\nabla_0 \overline{\omega (\mathbf{F_{t_0}^{t}(x_0)}, t)}},
 \label{eq: linearmomentum} 
 \end{equation}
 
 where $ \nu $ is the viscosity, $ \rho $ the density of the fluid, $ \mathbf{J} = \begin{pmatrix} 0 && 1 \\ -1 && 0 \end{pmatrix} $ and $ \mathbf{\omega} $ is the vorticity.
 
 For simplicity we can normalize equation $ \ref{eq: linearmomentum} $ by $ \nu $ and $ \rho $ and thus simply omit them for simplicity as they will play no role when exploring the structures of the field $ \mathbf{x'} $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| xt | array (51, Ny, Nx, 5)| X-meshgrid with Ny=Nx=1024 over which $ \omega(\mathbf{x}, t) $ is defined |
| yt | array (51, Ny, Nx, 5)| Y-meshgrid with Ny=Nx=1024 over which $ \omega(\mathbf{x}, t) $ is defined |
| omega | array (Ny, Nx, N) | $ \omega(\mathbf{x}, t) $ with $ N = 251 $ being the resolution in time |
| aux_grid | list (2,) | aux_grid[0]: dx_auxiliary spacing <br /> aux_grid[1]: dy_auxiliary spacing |
| Interpolant | list (2,) | Interpolant[0]: Interpolant for x-component of $ \mathbf{x}' $ <br /> Interpolant[1]: Interpolant for y-component of $ \mathbf{x}' $ |

In [ ]:
# import Rectangular bivariate spline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

# Import numpy 
import numpy as np

In [3]:
def InstantaneousActiveMomentum(xt, yt, omega):
    
    # auxiliary grid
    rho_x = aux_grid[0]
    rho_y = aux_grid[1]
    
    # iterate over all trajectories and compute omega along them
    
    # compute interpolant for omega over meshgrid X, Y    
    Interpolant_omega = RBS(Y[:,0], X[0,:], omega)
    
    Jgrad_omega = np.zeros((X.shape[0], X.shape[1], 2))
    
    for t in range(xt.shape[0]):
    
        # compute x'=J grad(omega), with J = [[0, 1], [-1, 0]]
        for i in range(X.shape[0]):
        
            for j in range(Y.shape[1]):
            
                # apply periodic boundary conditions to point 'x'
                x = np.array([X[i, j], Y[i, j]])
            
                # evaluate grad(omega) at 'x' using auxiliary grid
                xR = x[0] + rho_x
                OmegaR = Interpolant_omega(x[1], xR)[0][0]
            
                xL = x[0] - rho_x
                OmegaL = Interpolant_omega(x[1], xL)[0][0]
            
                xU = x[1] + rho_y
                OmegaU = Interpolant_omega(xU, x[0])[0][0]
            
                xD = x[1] - rho_y
                OmegaD = Interpolant_omega(xD, x[0])[0][0]
            
                Jgrad_omega[i, j, 0] = -(OmegaU - OmegaD)/(2*rho_y)
                Jgrad_omega[i, j, 1] = (OmegaR - OmegaL)/(2*rho_x)

    Interpolant = []
    Interpolant.append(RBS(Y[:,0], X[0,:], Jgrad_omega[:,:,0]))
    Interpolant.append(RBS(Y[:,0], X[0,:], Jgrad_omega[:,:,1]))
    
    return Interpolant